In [1]:
#impor packages
import pandas as pd
import numpy as np
import itertools
import os
import pickle
import json

In [2]:
input_path = "../datasets/raw/"

In [3]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'LABEVENTS.csv', dtype = {"SUBJECT_ID":"object",
                                                                        "HADM_ID":"object",
                                                                        "ITEMID":"object",
                                                                        "VALUE":"object",
                                                                        "VALUENUM":"object",
                                                                        "VALUEUOM":"object",
                                                                        "FLAG":"object",
                                                                        "CHARTTIME":"object"})
                 

In [4]:
# read D_LABITEMS.csv file
df_D_ITEMS = pd.read_csv(filepath_or_buffer=input_path+'D_LABITEMS.csv', dtype = {"ITEMID":"object"})

In [5]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 1)

In [6]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27854055 entries, 0 to 27854054
Data columns (total 8 columns):
SUBJECT_ID    27854055 non-null object
HADM_ID       22245034 non-null object
ITEMID        27854055 non-null object
CHARTTIME     27854055 non-null object
VALUE         27852810 non-null object
VALUENUM      24932835 non-null object
VALUEUOM      24753806 non-null object
FLAG          9844166 non-null object
dtypes: object(8)
memory usage: 1.7+ GB


In [7]:
df_D_ITEMS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 6 columns):
ROW_ID        753 non-null int64
ITEMID        753 non-null object
LABEL         753 non-null object
FLUID         753 non-null object
CATEGORY      753 non-null object
LOINC_CODE    585 non-null object
dtypes: int64(1), object(5)
memory usage: 35.4+ KB


In [8]:
#check contents of D_ITEMS
df_D_ITEMS

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN
5,551,51351,Lymphs,Cerebrospinal Fluid (CSF),Hematology,26479-6
6,552,51352,Macrophage,Cerebrospinal Fluid (CSF),Hematology,30426-1
7,553,51353,Mesothelial cells,Cerebrospinal Fluid (CSF),Hematology,30429-5
8,554,51354,Metamyelocytes,Cerebrospinal Fluid (CSF),Hematology,30366-9
9,555,51355,Monocytes,Cerebrospinal Fluid (CSF),Hematology,26486-1


In [9]:
#keep columns ITEMID and LABEL
df_D_ITEMS = df_D_ITEMS[['ITEMID','LABEL','FLUID','CATEGORY']]

In [10]:
df_D_ITEMS.head()

,ITEMID,LABEL,FLUID,CATEGORY
0,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology
1,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology
2,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology
3,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology
4,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology


In [11]:
df = pd.merge(left=df,right=df_D_ITEMS,how = 'left',on = 'ITEMID')

In [12]:
# check the merged dataframe
df.head(50)

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,LABEL,FLUID,CATEGORY
0,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN,pH,Blood,Blood Gas
1,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN,SPECIMEN TYPE,BLOOD,BLOOD GAS
2,3,NaN,50802,2101-10-12 18:17:00,-1,-1,mEq/L,NaN,Base Excess,Blood,Blood Gas
3,3,NaN,50804,2101-10-12 18:17:00,22,22,mEq/L,NaN,Calculated Total CO2,Blood,Blood Gas
4,3,NaN,50808,2101-10-12 18:17:00,0.93,.93,mmol/L,abnormal,Free Calcium,Blood,Blood Gas
5,3,NaN,50812,2101-10-12 18:17:00,NOT INTUBATED,NaN,NaN,NaN,Intubated,Blood,Blood Gas
6,3,NaN,50813,2101-10-12 18:17:00,1.8,1.8,mmol/L,NaN,Lactate,Blood,Blood Gas
7,3,NaN,50818,2101-10-12 18:17:00,33,33,mm Hg,NaN,pCO2,Blood,Blood Gas
8,3,NaN,50820,2101-10-12 18:17:00,7.42,7.42,units,NaN,pH,Blood,Blood Gas
9,3,NaN,50821,2101-10-12 18:17:00,80,80,mm Hg,NaN,pO2,Blood,Blood Gas


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27854055 entries, 0 to 27854054
Data columns (total 11 columns):
SUBJECT_ID    object
HADM_ID       object
ITEMID        object
CHARTTIME     object
VALUE         object
VALUENUM      object
VALUEUOM      object
FLAG          object
LABEL         object
FLUID         object
CATEGORY      object
dtypes: object(11)
memory usage: 2.5+ GB


In [15]:
df['VALUE_NUM_UOM'] = df['ITEMID']+'-'+df['VALUENUM']+'-'+df['VALUEUOM']

In [16]:
df.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG,LABEL,FLUID,CATEGORY,VALUE_NUM_UOM
0,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN,pH,Blood,Blood Gas,50820-7.39-units
1,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
2,3,NaN,50802,2101-10-12 18:17:00,-1,-1,mEq/L,NaN,Base Excess,Blood,Blood Gas,50802--1-mEq/L
3,3,NaN,50804,2101-10-12 18:17:00,22,22,mEq/L,NaN,Calculated Total CO2,Blood,Blood Gas,50804-22-mEq/L
4,3,NaN,50808,2101-10-12 18:17:00,0.93,.93,mmol/L,abnormal,Free Calcium,Blood,Blood Gas,50808-.93-mmol/L


In [17]:
df = df.drop(columns=['VALUE','VALUENUM','VALUEUOM','ITEMID'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27854055 entries, 0 to 27854054
Data columns (total 8 columns):
SUBJECT_ID       object
HADM_ID          object
CHARTTIME        object
FLAG             object
LABEL            object
FLUID            object
CATEGORY         object
VALUE_NUM_UOM    object
dtypes: object(8)
memory usage: 1.9+ GB


In [19]:
df = df.fillna('NaN')

In [20]:
df.head()

,SUBJECT_ID,HADM_ID,CHARTTIME,FLAG,LABEL,FLUID,CATEGORY,VALUE_NUM_UOM
0,3,NaN,2101-10-12 16:07:00,NaN,pH,Blood,Blood Gas,50820-7.39-units
1,3,NaN,2101-10-12 18:17:00,NaN,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN
2,3,NaN,2101-10-12 18:17:00,NaN,Base Excess,Blood,Blood Gas,50802--1-mEq/L
3,3,NaN,2101-10-12 18:17:00,NaN,Calculated Total CO2,Blood,Blood Gas,50804-22-mEq/L
4,3,NaN,2101-10-12 18:17:00,abnormal,Free Calcium,Blood,Blood Gas,50808-.93-mmol/L


In [22]:
df['labevents'] = df['LABEL']+' '+df['FLUID']+' '+df['CATEGORY']+' '+df['VALUE_NUM_UOM']+' '+df['FLAG']

In [23]:
df.head(10)

,SUBJECT_ID,HADM_ID,CHARTTIME,FLAG,LABEL,FLUID,CATEGORY,VALUE_NUM_UOM,labevents
0,3,NaN,2101-10-12 16:07:00,NaN,pH,Blood,Blood Gas,50820-7.39-units,pH Blood Blood Gas 50820-7.39-units NaN
1,3,NaN,2101-10-12 18:17:00,NaN,SPECIMEN TYPE,BLOOD,BLOOD GAS,NaN,SPECIMEN TYPE BLOOD BLOOD GAS NaN NaN
2,3,NaN,2101-10-12 18:17:00,NaN,Base Excess,Blood,Blood Gas,50802--1-mEq/L,Base Excess Blood Blood Gas 50802--1-mEq/L NaN
3,3,NaN,2101-10-12 18:17:00,NaN,Calculated Total CO2,Blood,Blood Gas,50804-22-mEq/L,Calculated Total CO2 Blood Blood Gas 50804-22-...
4,3,NaN,2101-10-12 18:17:00,abnormal,Free Calcium,Blood,Blood Gas,50808-.93-mmol/L,Free Calcium Blood Blood Gas 50808-.93-mmol/L ...
5,3,NaN,2101-10-12 18:17:00,NaN,Intubated,Blood,Blood Gas,NaN,Intubated Blood Blood Gas NaN NaN
6,3,NaN,2101-10-12 18:17:00,NaN,Lactate,Blood,Blood Gas,50813-1.8-mmol/L,Lactate Blood Blood Gas 50813-1.8-mmol/L NaN
7,3,NaN,2101-10-12 18:17:00,NaN,pCO2,Blood,Blood Gas,50818-33-mm Hg,pCO2 Blood Blood Gas 50818-33-mm Hg NaN
8,3,NaN,2101-10-12 18:17:00,NaN,pH,Blood,Blood Gas,50820-7.42-units,pH Blood Blood Gas 50820-7.42-units NaN
9,3,NaN,2101-10-12 18:17:00,NaN,pO2,Blood,Blood Gas,50821-80-mm Hg,pO2 Blood Blood Gas 50821-80-mm Hg NaN


In [24]:
df['labevents'] = df['labevents'].map(lambda x: x.split())

In [25]:
df = df.drop(labels=['FLAG','LABEL','FLUID','CATEGORY','VALUE_NUM_UOM'], axis = "columns")

In [26]:
# remove 'NaN' form all events
df['labevents'] = df['labevents'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [27]:
df.head(10)

,SUBJECT_ID,HADM_ID,CHARTTIME,labevents
0,3,NaN,2101-10-12 16:07:00,"[pH, Blood, Blood, Gas, 50820-7.39-units]"
1,3,NaN,2101-10-12 18:17:00,"[SPECIMEN, TYPE, BLOOD, BLOOD, GAS]"
2,3,NaN,2101-10-12 18:17:00,"[Base, Excess, Blood, Blood, Gas, 50802--1-mEq/L]"
3,3,NaN,2101-10-12 18:17:00,"[Calculated, Total, CO2, Blood, Blood, Gas, 50..."
4,3,NaN,2101-10-12 18:17:00,"[Free, Calcium, Blood, Blood, Gas, 50808-.93-m..."
5,3,NaN,2101-10-12 18:17:00,"[Intubated, Blood, Blood, Gas]"
6,3,NaN,2101-10-12 18:17:00,"[Lactate, Blood, Blood, Gas, 50813-1.8-mmol/L]"
7,3,NaN,2101-10-12 18:17:00,"[pCO2, Blood, Blood, Gas, 50818-33-mm, Hg]"
8,3,NaN,2101-10-12 18:17:00,"[pH, Blood, Blood, Gas, 50820-7.42-units]"
9,3,NaN,2101-10-12 18:17:00,"[pO2, Blood, Blood, Gas, 50821-80-mm, Hg]"


In [28]:
# create output path
mypath_output = "../datasets/labevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [29]:
# export the dataframe to JSON format
df.to_json(mypath_output+"labevents.json",orient = 'records') 